### tester différentes méthodes de detection de pic sur toute les masses, pour pouvoir tester DBSCAN et comparer les méthodes

In [ ]:
from skimage.feature import peak_local_max
from sklearn.cluster import DBSCAN
import skimage
import baseline_correction
from read_chroma import read_chromato_and_chromato_cube
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
import projection
import plot
import pandas as pd
import peak_detection

In [ ]:
# choose the methode
method = "peak_local_max"
# method = "persistence_homology"
# method = 'DOG'
# method = 'LOG'
# method = 'DOH'

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
# estimation with filtering_factor_estimation.ipynb
mod_time = 1.7
eps = 0.2

threshold_abs = 0.2

# estimation with estimation_persistence.ipynb
min_persitence = 0.02 # = threshold_relatif


In [ ]:
# filtering_factor = 50 #TODO
# sigma = skimage.restoration.estimate_sigma(chromato_tic_preprocessed, channel_axis=None)
# dynamic_threshold_fact = filtering_factor * sigma * 100 / np.max(chromato_tic_preprocessed)
# dynamic_threshold_fact

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)

In [ ]:
coordinates_all_mass = []
print("method =", method)

for mass in range(len(chromato_cube_preprocessed)):
    if method == "peak_local_max":
    # peak_detection.peak_local_max : mode = mass per masthout multiprocessing
    ## fonction plm_mas_per_mass_multiprocess()
        coord = skimage.feature.peak_local_max(chromato_cube_preprocessed[mass], min_distance=1, threshold_rel=threshold_abs)
        for x, y in coord:
            coordinates_all_mass.append([mass, x, y])


    if method == "persistence_homology":
    #persistance homology  without multiprocessing
        coord = peak_detection.pers_hom_kernel(
            None, m_chromato=chromato_cube_preprocessed[mass], 
            min_persistence=min_persitence,
            threshold_abs=threshold_abs)
        for x, y in coord:
            coordinates_all_mass.append([mass, x, y])
    
    if method == "DOG":
    #DOG without multiprocessing
        coord = peak_detection.blob_dog_kernel(
            None, m_chromato=chromato_cube_preprocessed[mass], 
            min_sigma=1, max_sigma=30, seuil=None,
            threshold_abs=threshold_abs,
            sigma_ratio=1.6)
        
        # ne supprime pas les doublons, et ne corrige pas r (sqrt(2) , r pas utile )
        for t1, t2, r in coord:
            coordinates_all_mass.append([mass, t1, t2])
    
    if method == "LOG":
    #LOG without multiprocessing
        coord = peak_detection.blob_log_kernel(
            None, m_chromato=chromato_cube_preprocessed[mass], 
            min_sigma=1, max_sigma=30, seuil=None,
            threshold_abs=threshold_abs,
            num_sigma=10)
        
        for t1, t2, r in coord:
            coordinates_all_mass.append([mass, t1, t2])

    if method == "DOH":
    #DOG without multiprocessing
        coord = peak_detection.blob_doh_kernel(
            None, m_chromato=chromato_cube_preprocessed[mass], 
            min_sigma=1, max_sigma=30, seuil=None,
            threshold_abs=threshold_abs,
            num_sigma=10)
        
        for t1, t2, r in coord:
            coordinates_all_mass.append([mass, t1, t2])
        


    


In [ ]:
# save to csv
name = file_path.split("/")[-1].split(".")[0]
df = pd.DataFrame(coordinates_all_mass, columns=["mass","x", "y"])
df.to_csv(f"figs/coord_{method}_{name}.csv", index=False)